In [1]:
from keras_unet_collection import models

2023-04-23 11:58:41.396530: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 11:58:41.508795: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-23 11:58:41.515387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 11:58:43.346262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
import numpy as np
import skimage.io as io
import skimage.transform as trans
from keras.models import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow as tf

In [3]:
IMAGE_PATH = "./train/images/"
MASK_PATH = "./train/gt/"
READ_SIZE = (512, 512)
INPUT_SIZE = (512, 512, 3)
BATCH_SIZE = 32
VAL_SPLIT = 0.2

In [16]:
def preprocess_input(img):
    return img/255

def process_mask(img):
    mask = img.astype(np.float32)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    return mask

def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def IoU_coeff(y_true, y_pred):
    axes = (1,2) 
    intersection = tf.keras.backend.sum(tf.math.abs(y_pred * y_true), axis=axes) 
    mask = tf.keras.backend.sum(tf.math.abs(y_true), axis=axes) + tf.keras.backend.sum(tf.math.abs(y_pred), axis=axes)
    union = mask - intersection
    smooth = .001
    iou = (intersection + smooth) / (union + smooth)
    return iou

checkpoint_filepath = 'unet-1{epoch:02d}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='loss',
    mode='min',
    save_freq='epoch',
    save_best_only=True)

In [17]:
image_data = ImageDataGenerator(preprocessing_function=preprocess_input ,validation_split=VAL_SPLIT)
mask_data = ImageDataGenerator(preprocessing_function=process_mask, validation_split=VAL_SPLIT)

image_gen = image_data.flow_from_directory(IMAGE_PATH, target_size=READ_SIZE, class_mode=None, 
                                           batch_size=BATCH_SIZE, shuffle=False, subset='training')
val_image_gen = image_data.flow_from_directory(IMAGE_PATH, target_size=READ_SIZE, class_mode=None, 
                                           batch_size=BATCH_SIZE, shuffle=False, subset='validation')
mask_gen = image_data.flow_from_directory(MASK_PATH, target_size=READ_SIZE, class_mode=None, 
                                          batch_size=BATCH_SIZE, shuffle=False, subset='training')
val_mask_gen = image_data.flow_from_directory(MASK_PATH, target_size=READ_SIZE, class_mode=None, 
                                          batch_size=BATCH_SIZE, shuffle=False, subset='validation')

train_generator = zip(image_gen, mask_gen)
val_generator = zip(val_image_gen, val_mask_gen)


Found 576 images belonging to 1 classes.
Found 144 images belonging to 1 classes.
Found 576 images belonging to 1 classes.
Found 144 images belonging to 1 classes.


In [12]:
model = models.att_unet_2d(INPUT_SIZE, filter_num=[64, 128, 256, 512, 1024], n_labels=1, 
                           stack_num_down=2, stack_num_up=2, activation='ReLU', 
                           atten_activation='ReLU', attention='add', output_activation='Sigmoid', 
                           batch_norm=True, pool=False, unpool=False, 
                           backbone='VGG16', weights='imagenet', 
                           freeze_backbone=True, freeze_batch_norm=True, 
                           name='attunet')

In [13]:
model.compile(optimizer=Adam(1e-3, decay=1e-6), loss=dice_coef_loss, metrics=[dice_coef, IoU_coeff])

In [ ]:
history = model.fit(train_generator, steps_per_epoch = image_gen.samples // BATCH_SIZE, 
                    validation_data = val_generator, validation_steps = val_image_gen.samples // BATCH_SIZE, 
                    epochs=10, callbacks=[model_checkpoint_callback])

Epoch 1/10


2023-04-23 19:56:17.086939: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


18/18 [==============================] - ETA: 0s - loss: 0.0165 - dice_coef: 0.9835 - IoU_coeff: 0.3576  

2023-04-23 20:33:22.061682: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


18/18 [==============================] - 2355s 131s/step - loss: 0.0165 - dice_coef: 0.9835 - IoU_coeff: 0.3576 - val_loss: 0.0282 - val_dice_coef: 0.9718 - val_IoU_coeff: 0.3853
Epoch 2/10
18/18 [==============================] - 2316s 129s/step - loss: 0.0134 - dice_coef: 0.9866 - IoU_coeff: 0.3636 - val_loss: 0.0385 - val_dice_coef: 0.9615 - val_IoU_coeff: 0.3612
Epoch 3/10
18/18 [==============================] - 2336s 130s/step - loss: 0.0118 - dice_coef: 0.9882 - IoU_coeff: 0.3698 - val_loss: 0.0522 - val_dice_coef: 0.9478 - val_IoU_coeff: 0.3344
Epoch 4/10
18/18 [==============================] - 2327s 130s/step - loss: 0.0117 - dice_coef: 0.9883 - IoU_coeff: 0.3734 - val_loss: 0.0590 - val_dice_coef: 0.9410 - val_IoU_coeff: 0.3165
Epoch 5/10
18/18 [==============================] - 2343s 131s/step - loss: 0.0095 - dice_coef: 0.9905 - IoU_coeff: 0.3837 - val_loss: 0.0567 - val_dice_coef: 0.9433 - val_IoU_coeff: 0.3247
Epoch 6/10
17/18 [===========================>..] - ETA: 2:02

In [15]:
import pickle
model_json = model.to_json()
with open("kunetmodel-2.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("kunetmodel.h5")
with open('./kunetHistory-1', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
#Load Trained Model for evaluation

In [ ]:
json_file = open('kunetmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("unet-101.hdf5")

In [ ]:
import tifffile as tiff
img = val_gen.next()[0]
tiff.imshow(img)

In [ ]:
mask = model.predict(np.array([img]))
tiff.imshow(mask)

In [ ]:
def get_boundingbox(mask):
    mask = (mask[0]).astype(int)
    ret, thresh = cv2.threshold((mask*255).astype(np.uint8), 127, 255, cv2.THRESH_BINARY)
    
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Get bounding boxes for each contour
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append((x, y, w, h))

    # Print the bounding boxes
    return bounding_boxes

In [ ]:
img = img * 255
for box in boxes:
    x,y,w,h = box
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 1)
tiff.imshow(img)